# Aerial Imagery Classifier

Sam Blake, started 23 December 2022

In [3]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook
%pylab notebook

import os
import time
import glob
import math
import numpy as np
import cv2
import matplotlib as mpl
from matplotlib import pyplot as plt

from tqdm import tqdm

import numba
from numba import jit, prange, set_num_threads

from PIL import Image

import rasterio
from rasterio.plot import show

from aerial_imagery_classifier import imagery_statistical_binary_classification, \
    show_bgr_image, show_bgra_image, show_greyscale_image, denoise 

plt.ioff()
plt.rcParams["figure.figsize"] = (10, 6)

Populating the interactive namespace from numpy and matplotlib


## De Bortolli Rice

In [52]:
tile_42_crop_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/crop_1.png')
tile_42_dirt_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/dirt_1.png')
tile_42_water_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/water_1.png')
tile_42_weed_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_42/weed_1.png')

In [53]:
tile_47_crop_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_1.png')
tile_47_crop_2 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_2.png')
tile_47_crop_3 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/crop_3.png')
tile_47_dead_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/dead_1.png')
tile_47_weed_1 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/weed_1.png')
tile_47_weed_2 = cv2.imread('../de_bortolli/DeBortolli/map/spectra/tile_47/weed_2.png')

In [59]:
filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_42.tif'
filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_47.tif'
# filename = '../de_bortolli/DeBortolli/map/tiles/result_tile_52.tif'

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [tile_47_weed_1, tile_47_weed_2, tile_42_weed_1], \
        [tile_47_crop_1, tile_47_crop_2, tile_47_crop_3, tile_47_dead_1, \
            tile_42_dirt_1, tile_42_water_1], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (7,7), \
        template_blur_kernel_size = (3,3), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 1, \
        common_tolerance = 1, \
        n_sigma_thresholds = [2.75, 3], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 175, morph_close = 10, \
        dilation_size = 301, \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)


# Export spray map to GeoTIFF.
output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, 255*spray_region)

# Export weed map to GeoTIFF.
output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

# Export classification CI to GeoTIFF.
output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, classification_confidence)

In [58]:
file_pattern = '/Users/user/Documents/Research/rotavision/de_bortolli/DeBortolli/map/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [tile_47_weed_1, tile_47_weed_2, tile_42_weed_1], \
            [tile_47_crop_1, tile_47_crop_2, tile_47_crop_3, tile_47_dead_1, \
                tile_42_dirt_1, tile_42_water_1], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (7,7), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 1, \
            common_tolerance = 1, \
            n_sigma_thresholds = [2.75, 3], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 175, morph_close = 10, \
            dilation_size = 301, \
            half_normal_dist = True, \
            export_histogram = False, \
            export_heatmap = False, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, 255*spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

result_tile_34.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0009
   31762 [px] in 2.75 CI, 0.0411 [%] of all pixels. 0.0411 [%] cumulative total.
   15995 [px] in 3.00 CI, 0.0207 [%] of all pixels. 0.0618 [%] cumulative total.
Sample not detected in 99.94 [%] of all pixels.
Spray region is 2.42 [%] of total region.
result_tile_20.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0015
   61434 [px] in 2.75 CI, 0.0795 [%] of all pixels. 0.0795 [%] cumulative total.
   27073 [px] in 3.00 CI, 0.0350 [%] of all pixels. 0.1146 [%] cumulative total.
Sample not detected in 99.89 [%] of all pixels.
Spray region is 4.22 [%] of total region.
result_tile_21.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0014
    5592 [px] in 2.75 CI, 0.0072 [%] of all pixels. 0.0072 [%] cumulative

Spray region is 2.47 [%] of total region.
result_tile_56.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0010
   38280 [px] in 2.75 CI, 0.0496 [%] of all pixels. 0.0496 [%] cumulative total.
   12385 [px] in 3.00 CI, 0.0160 [%] of all pixels. 0.0656 [%] cumulative total.
Sample not detected in 99.93 [%] of all pixels.
Spray region is 2.82 [%] of total region.
result_tile_40.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0017
  301171 [px] in 2.75 CI, 0.3899 [%] of all pixels. 0.3899 [%] cumulative total.
  119555 [px] in 3.00 CI, 0.1548 [%] of all pixels. 0.5446 [%] cumulative total.
Sample not detected in 99.46 [%] of all pixels.
Spray region is 13.38 [%] of total region.
result_tile_54.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 

Spray region is 2.57 [%] of total region.
result_tile_6.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0014
  146194 [px] in 2.75 CI, 0.1892 [%] of all pixels. 0.1892 [%] cumulative total.
  119483 [px] in 3.00 CI, 0.1547 [%] of all pixels. 0.3439 [%] cumulative total.
Sample not detected in 99.66 [%] of all pixels.
Spray region is 8.24 [%] of total region.
result_tile_61.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0010
   34692 [px] in 2.75 CI, 0.0449 [%] of all pixels. 0.0449 [%] cumulative total.
   12151 [px] in 3.00 CI, 0.0157 [%] of all pixels. 0.0606 [%] cumulative total.
Sample not detected in 99.94 [%] of all pixels.
Spray region is 1.65 [%] of total region.
result_tile_49.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0016
  119302 [px] in 2.75 CI, 0.1544

Spray region is 0.00 [%] of total region.
result_tile_14.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0008
   91240 [px] in 2.75 CI, 0.1181 [%] of all pixels. 0.1181 [%] cumulative total.
   50856 [px] in 3.00 CI, 0.0658 [%] of all pixels. 0.1839 [%] cumulative total.
Sample not detected in 99.82 [%] of all pixels.
Spray region is 7.56 [%] of total region.
result_tile_16.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0011
  126774 [px] in 2.75 CI, 0.1641 [%] of all pixels. 0.1641 [%] cumulative total.
   53874 [px] in 3.00 CI, 0.0697 [%] of all pixels. 0.2338 [%] cumulative total.
Sample not detected in 99.77 [%] of all pixels.
Spray region is 12.12 [%] of total region.
result_tile_17.tif
number of distinct spectra in sample A is 2645
number of distinct spectra in sample B is 6540
mean = 0.0000, std = 0.0016
  109605 [px] in 2.75 CI, 0.14

## TavistockAG Precision Weed Trial

In [31]:
dry_crop_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_1.jpg')
dry_crop_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_2.jpg')
dry_crop_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_3.jpg')
dry_crop_4 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_4.jpg')
dry_crop_5 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_5.jpg')
dry_crop_6 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_6.jpg')
dry_crop_7 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_7.jpg')

dry_crop_8 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/crop_8.jpg')
rocks_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_1.jpg')
rocks_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_2.jpg')
rocks_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_3.jpg')
rocks_4 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_4.jpg')
rocks_5 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/rocks_5.jpg')

In [32]:
weed_1 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_1.jpg')
weed_2 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_2.jpg')
weed_3 = cv2.imread('../TavistockAG_Precision_Weed_Trial/spectra/weed_3.jpg')

In [138]:
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_399.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_446.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_658.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_383.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_607.tif'
filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1150.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1151.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1141.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1136.tif'
# filename = '../TavistockAG_Precision_Weed_Trial/calibration/100_Orthomosaic_rgb_tile_1137.tif'

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [weed_1, weed_2, weed_3], \
        [dry_crop_1, dry_crop_2, dry_crop_3, dry_crop_4, dry_crop_5, dry_crop_6, dry_crop_7], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (7,7), \
        template_blur_kernel_size = (3,3), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 1, \
        common_tolerance = 1, \
        n_sigma_thresholds = [2.5, 2.6, 2.7, 2.8, 2.9, 3], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 175, morph_close = 10, \
        dilation_size = 301, \
        export_histogram = True, \
        export_heatmap = True, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)


# Export spray map to GeoTIFF.
output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, 255*spray_region)

# Export weed map to GeoTIFF.
output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

# Export classification CI to GeoTIFF.
output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

kwargs = dataset.meta
kwargs.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw')

with rasterio.open(output_filename, 'w', **kwargs) as dst:
    dst.write_band(1, classification_confidence)

In [82]:
# TODO: export json or yaml file of input parameterisations.

In [ ]:
file_pattern = '/Users/user/Documents/Research/rotavision/TavistockAG_Precision_Weed_Trial/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [weed_1, weed_2, weed_3], \
            [dry_crop_1, dry_crop_2, dry_crop_3, dry_crop_4, dry_crop_5, dry_crop_6, dry_crop_7], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (7,7), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 1, \
            common_tolerance = 1, \
            n_sigma_thresholds = [2.5, 2.75, 3], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 175, morph_close = 10, \
            dilation_size = 301, \
            export_histogram = False, \
            export_heatmap = False, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

## nihill radish

Import features for detection: 

In [12]:
nihill_radish_sample_1 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_1.jpg')
nihill_radish_sample_2 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_2.jpg')
nihill_radish_sample_3 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_3.jpg')
nihill_radish_sample_4 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_4.jpg')
nihill_radish_sample_5 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_5.jpg')
nihill_radish_sample_6 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_6.jpg')
nihill_radish_sample_7 = cv2.imread('../nihill_radish/spectra/nihill_radish_sample_7.jpg')

nihill_radish_sample_1.shape,\
nihill_radish_sample_2.shape,\
nihill_radish_sample_3.shape,\
nihill_radish_sample_4.shape,\
nihill_radish_sample_5.shape,\
nihill_radish_sample_6.shape,\
nihill_radish_sample_7.shape

((243, 552, 3),
 (68, 69, 3),
 (45, 69, 3),
 (62, 29, 3),
 (81, 59, 3),
 (109, 95, 3),
 (105, 129, 3))

In [8]:
wheat_sample_1 = cv2.imread('../nihill_radish/spectra/nihill_crop_sample_1.jpg')
wheat_sample_2 = cv2.imread('../nihill_radish/spectra/nihill_crop_sample_2.jpg')
wheat_sample_3 = cv2.imread('../nihill_radish/spectra/nihill_crop_sample_3.jpg')
wheat_sample_4 = cv2.imread('../nihill_radish/spectra/nihill_crop_sample_4.jpg')
wheat_sample_1.shape,\
wheat_sample_2.shape,\
wheat_sample_3.shape,\
wheat_sample_4.shape

((413, 942, 3), (205, 796, 3), (547, 679, 3), (963, 1149, 3))

In [10]:
wheat_dead_sample_1 = cv2.imread('../nihill_radish/spectra/nihill_crop_dead_sample_1.jpg')
wheat_dead_sample_2 = cv2.imread('../nihill_radish/spectra/nihill_crop_dead_sample_2.jpg')
wheat_dead_sample_3 = cv2.imread('../nihill_radish/spectra/nihill_crop_dead_sample_3.jpg')
wheat_dead_sample_1.shape,\
wheat_dead_sample_2.shape,\
wheat_dead_sample_3.shape

((575, 985, 3), (563, 619, 3), (572, 2050, 3))

In [ ]:
wheat_dead_sample_1, wheat_dead_sample_2, wheat_dead_sample_3

In [ ]:
filename = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/White_Radish_Orthomosaic_tile_9.tif'
filename = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/White_Radish_Orthomosaic_tile_47.tif'
filename = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/White_Radish_Orthomosaic_tile_26.tif'
filename = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/White_Radish_Orthomosaic_tile_24.tif'

dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [nihill_radish_sample_1], \
        [wheat_sample_1, wheat_sample_2, wheat_sample_3, wheat_sample_4], \
        image_resize_ratio = 10, \
        image_blur_kernel_size = (19,19), \
        template_blur_kernel_size = (7,7), \
        template_resize_size = (64,64), \
        duplicate_tolerance = 5, \
        common_tolerance = 1, \
        n_sigma_thresholds = [2.75, 3], \
        equalise = True, \
        denoise_classification_map = True, morph_open = 175, morph_close = 20, \
        dilation_size = 100, \
        limit_outliers = False, \
        half_normal_dist = True, \
        export_histogram = False, \
        export_heatmap = False, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)

In [29]:
file_pattern = '/Users/user/Documents/Research/rotavision/nihill_radish/tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    dataset = rasterio.open(filename)

    red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [nihill_radish_sample_1], \
            [wheat_sample_1, wheat_sample_2, wheat_sample_3, wheat_sample_4], \
            image_resize_ratio = 10, \
            image_blur_kernel_size = (19,19), \
            template_blur_kernel_size = (7,7), \
            template_resize_size = (64,64), \
            duplicate_tolerance = 5, \
            common_tolerance = 1, \
            n_sigma_thresholds = [2.75, 3], \
            equalise = True, \
            denoise_classification_map = True, morph_open = 175, morph_close = 20, \
            dilation_size = 100, \
            limit_outliers = False, \
            half_normal_dist = True, \
            export_histogram = False, \
            export_heatmap = False, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)


    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, 255*spray_region)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

White_Radish_Orthomosaic_tile_28.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449


/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
White_Radish_Orthomosaic_tile_14.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
White_Radish_Orthomosaic_tile_15.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = 0.0019
  244390 [px] in 2.75 CI, 0.5731 [%] of all pixels. 0.5731 [%] cumulative total.
  200440 [px] in 3.00 CI, 0.4700 [%] of all pixels. 1.0431 [%] cumulative tot

Spray region is 0.00 [%] of total region.
White_Radish_Orthomosaic_tile_49.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
White_Radish_Orthomosaic_tile_59.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = nan
       0 [px] in 2.75 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
White_Radish_Orthomosaic_tile_2.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, s

White_Radish_Orthomosaic_tile_8.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = 0.0028
  268820 [px] in 2.75 CI, 0.6304 [%] of all pixels. 0.6304 [%] cumulative total.
  231099 [px] in 3.00 CI, 0.5419 [%] of all pixels. 1.1723 [%] cumulative total.
Sample not detected in 98.83 [%] of all pixels.
Spray region is 2.92 [%] of total region.
White_Radish_Orthomosaic_tile_9.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = 0.0027
  526235 [px] in 2.75 CI, 1.2340 [%] of all pixels. 1.2340 [%] cumulative total.
  444750 [px] in 3.00 CI, 1.0430 [%] of all pixels. 2.2770 [%] cumulative total.
Sample not detected in 97.72 [%] of all pixels.
Spray region is 6.68 [%] of total region.
White_Radish_Orthomosaic_tile_46.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = 0.0030
  233356 [px] in 2.75 CI, 0

number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = 0.0020
  397157 [px] in 2.75 CI, 0.9314 [%] of all pixels. 0.9314 [%] cumulative total.
  315644 [px] in 3.00 CI, 0.7402 [%] of all pixels. 1.6716 [%] cumulative total.
Sample not detected in 98.33 [%] of all pixels.
Spray region is 6.23 [%] of total region.
White_Radish_Orthomosaic_tile_31.tif
number of distinct spectra in sample A is 259
number of distinct spectra in sample B is 449
mean = 0.0000, std = 0.0017
  289794 [px] in 2.75 CI, 0.6796 [%] of all pixels. 0.6796 [%] cumulative total.
  244472 [px] in 3.00 CI, 0.5733 [%] of all pixels. 1.2529 [%] cumulative total.
Sample not detected in 98.75 [%] of all pixels.
Spray region is 3.80 [%] of total region.


## 230324_NoelsPdk

In [83]:
directory = '/Users/user/Documents/weedmap/trials/230324 Noels Pdk Melons 1GSD M3E/'

In [55]:
weed_7 = cv2.imread(directory + 'spectra/weed_7.jpg')
weed_7.shape

(228, 219, 3)

In [ ]:
weed_7[weed_7[:,:,1] > 150] = [255,255,255]

plt.imshow(weed_7[:,:,::-1])
plt.show()
# cv2.imwrite(directory + 'weed_7_pre.jpg', weed_7)

In [23]:
weed_8 = cv2.imread(directory + 'spectra/weed_8.jpg')
weed_8.shape

(492, 537, 3)

In [ ]:
weed_8[weed_8[:,:,1] > 150] = [255,255,255]

plt.imshow(weed_8[:,:,::-1])
plt.show()
# cv2.imwrite(directory + 'weed_8_pre.jpg', weed_8)

In [75]:
weed_15 = cv2.imread(directory + 'spectra/weed_15.jpg')
weed_15.shape

(145, 236, 3)

In [ ]:
weed_15[weed_15[:,:,1] > 120] = [255,255,255]

plt.imshow(weed_15[:,:,::-1])
plt.show()
cv2.imwrite(directory + 'spectra/weed_15_pre.jpg', weed_15)

In [922]:
weed_1 = cv2.imread(directory + 'spectra/weed_1.jpg')
weed_2 = cv2.imread(directory + 'spectra/weed_2.jpg')
weed_3 = cv2.imread(directory + 'spectra/weed_3.jpg')
weed_4 = cv2.imread(directory + 'spectra/weed_4.jpg')
weed_5 = cv2.imread(directory + 'spectra/weed_5.jpg')
weed_6 = cv2.imread(directory + 'spectra/weed_6.jpg')
weed_7 = cv2.imread(directory + 'spectra/weed_7.jpg')
weed_8 = cv2.imread(directory + 'spectra/weed_8.jpg')
weed_9 = cv2.imread(directory + 'spectra/weed_9.jpg')

In [923]:
crop_1 = cv2.imread(directory + 'spectra/crop_1.jpg')
crop_2 = cv2.imread(directory + 'spectra/crop_2.jpg')
crop_3 = cv2.imread(directory + 'spectra/crop_3.jpg')
crop_4 = cv2.imread(directory + 'spectra/crop_4.jpg')
crop_5 = cv2.imread(directory + 'spectra/crop_5.jpg')
crop_6 = cv2.imread(directory + 'spectra/crop_6.jpg')
crop_7 = cv2.imread(directory + 'spectra/crop_7.jpg')
crop_8 = cv2.imread(directory + 'spectra/crop_8.jpg')
crop_9 = cv2.imread(directory + 'spectra/crop_9.jpg')
crop_10 = cv2.imread(directory + 'spectra/crop_10.jpg')
crop_11 = cv2.imread(directory + 'spectra/crop_11.jpg')
crop_12 = cv2.imread(directory + 'spectra/crop_12.jpg')
crop_13 = cv2.imread(directory + 'spectra/crop_13.jpg')
crop_14 = cv2.imread(directory + 'spectra/crop_14.jpg')

In [976]:
filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_124_EQUALISED.tif'
filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_250_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_244_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_242_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_192_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_185_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_134_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_115_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_88_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_145_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_207_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_197_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_193_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_145_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_89_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_246_EQUALISED.tif'
# filename = directory + 'tiles/230324_NoelsPdk_75pc_tile_233_EQUALISED.tif'

In [977]:
test = cv2.imread(filename)
test.shape

(7077, 5912, 3)

In [877]:
from ndvi_feature_imagery_classifier import green_red_ratio_filter_threshold

In [ ]:
weed_map, spray_map = green_red_ratio_filter_threshold(
    test, n_sigma = 2., 
    plotting = True, verbose = True)

In [ ]:
file_pattern = directory + 'tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'WEED_MAP' in filename or 'SPRAY_MAP' in filename:
        continue
    
    try:
        dataset = rasterio.open(filename)
        red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    except:
        continue
        
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    weed_map, spray_map = green_red_ratio_filter_threshold(img, n_sigma = 3., verbose = True)

    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, spray_map)

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, weed_map)

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [weed_1, weed_2, weed_3, weed_4, weed_5, weed_6, weed_7, \
        weed_8, weed_9], \
        [crop_1, crop_2, crop_3, crop_4, crop_5, crop_6, crop_7, crop_8, \
        crop_9, crop_10, crop_11, crop_12], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (7,7), \
        template_blur_kernel_size = (3,3), \
        template_resize_size = 64, \
        duplicate_tolerance = 5, \
        common_tolerance = 5, \
        n_sigma_thresholds = [2.], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 100, morph_close = 8, \
        dilation_size = 151, \
        limit_outliers = False, \
        export_histogram = False, \
        export_heatmap = False, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)

In [979]:
file_pattern = directory + 'tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    try:
        dataset = rasterio.open(filename)
        red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    except:
        continue
        
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [weed_1, weed_2, weed_3, weed_4, weed_5, weed_6, weed_7, \
            weed_8, weed_9], \
            [crop_1, crop_2, crop_3, crop_4, crop_5, crop_6, crop_7, crop_8, \
            crop_9, crop_10, crop_11, crop_12], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (7,7), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = 64, \
            duplicate_tolerance = 5, \
            common_tolerance = 5, \
            n_sigma_thresholds = [2.], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 100, morph_close = 8, \
            dilation_size = 151, \
            z_score = False, \
            limit_outliers = False, \
            export_histogram = False, \
            export_heatmap = False, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)

    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(spray_region == 0, 0, 255))

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

230324_NoelsPdk_75pc_tile_268_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430


/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


mean = 0.0000, std = nan
       0 [px] in 2.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
230324_NoelsPdk_75pc_tile_164_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.8858
  151367 [px] in 2.00 CI, 0.3618 [%] of all pixels. 0.3618 [%] cumulative total.
Sample not detected in 99.64 [%] of all pixels.
Spray region is 9.00 [%] of total region.
230324_NoelsPdk_75pc_tile_173_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 2.1216
  144484 [px] in 2.00 CI, 0.3453 [%] of all pixels. 0.3453 [%] cumulative total.
Sample not detected in 99.65 [%] of all pixels.
Spray region is 7.67 [%] of total region.
230324_NoelsPdk_75pc_tile_251_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B i

Spray region is 5.47 [%] of total region.
230324_NoelsPdk_75pc_tile_110_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.1416
    3821 [px] in 2.00 CI, 0.0091 [%] of all pixels. 0.0091 [%] cumulative total.
Sample not detected in 99.99 [%] of all pixels.
Spray region is 1.03 [%] of total region.
230324_NoelsPdk_75pc_tile_107_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.0155
   12775 [px] in 2.00 CI, 0.0305 [%] of all pixels. 0.0305 [%] cumulative total.
Sample not detected in 99.97 [%] of all pixels.
Spray region is 1.42 [%] of total region.
230324_NoelsPdk_75pc_tile_78_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = nan
       0 [px] in 2.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pix

mean = 0.0000, std = 1.5977
   96093 [px] in 2.00 CI, 0.2297 [%] of all pixels. 0.2297 [%] cumulative total.
Sample not detected in 99.77 [%] of all pixels.
Spray region is 6.64 [%] of total region.
230324_NoelsPdk_75pc_tile_180_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.8780
  228164 [px] in 2.00 CI, 0.5453 [%] of all pixels. 0.5453 [%] cumulative total.
Sample not detected in 99.45 [%] of all pixels.
Spray region is 16.62 [%] of total region.
230324_NoelsPdk_75pc_tile_226_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.0771
    9563 [px] in 2.00 CI, 0.0229 [%] of all pixels. 0.0229 [%] cumulative total.
Sample not detected in 99.98 [%] of all pixels.
Spray region is 1.46 [%] of total region.
230324_NoelsPdk_75pc_tile_55_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B

...

Spray region is 0.00 [%] of total region.
230324_NoelsPdk_75pc_tile_259_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = nan
       0 [px] in 2.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
230324_NoelsPdk_75pc_tile_97_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.0435
    9468 [px] in 2.00 CI, 0.0226 [%] of all pixels. 0.0226 [%] cumulative total.
Sample not detected in 99.98 [%] of all pixels.
Spray region is 1.99 [%] of total region.
230324_NoelsPdk_75pc_tile_155_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 0.9262
    3881 [px] in 2.00 CI, 0.0093 [%] of all pixels. 0.0093 [%] cumulative total.
Sample not detected in 99.99 [%] of all pix

mean = 0.0000, std = 0.8914
    6233 [px] in 2.00 CI, 0.0149 [%] of all pixels. 0.0149 [%] cumulative total.
Sample not detected in 99.99 [%] of all pixels.
Spray region is 0.97 [%] of total region.
230324_NoelsPdk_75pc_tile_49_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.3179
  104488 [px] in 2.00 CI, 0.2497 [%] of all pixels. 0.2497 [%] cumulative total.
Sample not detected in 99.75 [%] of all pixels.
Spray region is 9.15 [%] of total region.
230324_NoelsPdk_75pc_tile_121_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.4977
   62476 [px] in 2.00 CI, 0.1493 [%] of all pixels. 0.1493 [%] cumulative total.
Sample not detected in 99.85 [%] of all pixels.
Spray region is 5.28 [%] of total region.
230324_NoelsPdk_75pc_tile_70_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B i

Spray region is 1.07 [%] of total region.
230324_NoelsPdk_75pc_tile_116_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = nan
       0 [px] in 2.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
230324_NoelsPdk_75pc_tile_69_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.4939
   23414 [px] in 2.00 CI, 0.0560 [%] of all pixels. 0.0560 [%] cumulative total.
Sample not detected in 99.94 [%] of all pixels.
Spray region is 2.37 [%] of total region.
230324_NoelsPdk_75pc_tile_240_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.1833
    2970 [px] in 2.00 CI, 0.0071 [%] of all pixels. 0.0071 [%] cumulative total.
Sample not detected in 99.99 [%] of all pix

mean = 0.0000, std = 1.2558
    1487 [px] in 2.00 CI, 0.0036 [%] of all pixels. 0.0036 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.11 [%] of total region.
230324_NoelsPdk_75pc_tile_247_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = nan
       0 [px] in 2.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
230324_NoelsPdk_75pc_tile_34_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.3972
   74816 [px] in 2.00 CI, 0.1788 [%] of all pixels. 0.1788 [%] cumulative total.
Sample not detected in 99.82 [%] of all pixels.
Spray region is 6.04 [%] of total region.
230324_NoelsPdk_75pc_tile_23_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is

Spray region is 4.53 [%] of total region.
230324_NoelsPdk_75pc_tile_219_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 2.1367
  314220 [px] in 2.00 CI, 0.7510 [%] of all pixels. 0.7510 [%] cumulative total.
Sample not detected in 99.25 [%] of all pixels.
Spray region is 14.69 [%] of total region.
230324_NoelsPdk_75pc_tile_115_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 1.1485
    2880 [px] in 2.00 CI, 0.0069 [%] of all pixels. 0.0069 [%] cumulative total.
Sample not detected in 99.99 [%] of all pixels.
Spray region is 0.21 [%] of total region.
230324_NoelsPdk_75pc_tile_102_EQUALISED.tif
number of distinct spectra in sample A is 1257
number of distinct spectra in sample B is 430
mean = 0.0000, std = 0.5367
    2438 [px] in 2.00 CI, 0.0058 [%] of all pixels. 0.0058 [%] cumulative total.
Sample not detected in 99.99 [%] of all

## Whealby West Melons

In [980]:
directory = '/Users/user/Documents/weedmap/trials/230324 Whealby West Melons 1GSD M3E/'

In [982]:
weed_1 = cv2.imread(directory + 'spectra/weed_1.jpg')
weed_2 = cv2.imread(directory + 'spectra/weed_2.jpg')
weed_3 = cv2.imread(directory + 'spectra/weed_3.jpg')
weed_4 = cv2.imread(directory + 'spectra/weed_4.jpg')
weed_5 = cv2.imread(directory + 'spectra/weed_5.jpg')
weed_6 = cv2.imread(directory + 'spectra/weed_6.jpg')
weed_7 = cv2.imread(directory + 'spectra/weed_7.jpg')
weed_8 = cv2.imread(directory + 'spectra/weed_8.jpg')
weed_9 = cv2.imread(directory + 'spectra/weed_9.jpg')
weed_10 = cv2.imread(directory + 'spectra/weed_10.jpg')
weed_11 = cv2.imread(directory + 'spectra/weed_11.jpg')
weed_12 = cv2.imread(directory + 'spectra/weed_12.jpg')

In [981]:
crop_1 = cv2.imread(directory + 'spectra/crop_1.jpg')
crop_2 = cv2.imread(directory + 'spectra/crop_2.jpg')
crop_3 = cv2.imread(directory + 'spectra/crop_3.jpg')
crop_4 = cv2.imread(directory + 'spectra/crop_4.jpg')
crop_5 = cv2.imread(directory + 'spectra/crop_5.jpg')
crop_6 = cv2.imread(directory + 'spectra/crop_6.jpg')
crop_7 = cv2.imread(directory + 'spectra/crop_7.jpg')
crop_8 = cv2.imread(directory + 'spectra/crop_8.jpg')
crop_9 = cv2.imread(directory + 'spectra/crop_9.jpg')
crop_10 = cv2.imread(directory + 'spectra/crop_10.jpg')

In [994]:
filename = directory + 'tiles/260324_Whealby_Wst_75pc_tile_113_EQUALISED.tif'
# filename = directory + 'tiles/260324_Whealby_Wst_75pc_tile_30_EQUALISED.tif'
test = cv2.imread(filename)
test.shape

(7184, 11278, 3)

In [ ]:
def windowed_median_std(array, window_size):
    

In [ ]:
dataset = rasterio.open(filename)

red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
img[:,:,0] = blue
img[:,:,1] = green
img[:,:,2] = red

density, classification_image, classification_confidence, spray_region = \
    imagery_statistical_binary_classification(\
        img, \
        [weed_1, weed_2, weed_3, weed_4, weed_5, weed_6, weed_7, \
        weed_8, weed_9, weed_10, weed_11, weed_12], \
        [crop_1, crop_2, crop_3, crop_4, crop_5, crop_6, crop_7, crop_8, \
        crop_9, crop_10], \
        image_resize_ratio = 5, \
        image_blur_kernel_size = (17,17), \
        template_blur_kernel_size = (3,3), \
        template_resize_size = 64, \
        duplicate_tolerance = 5, \
        common_tolerance = 5, \
        n_sigma_thresholds = [3.], \
        equalise = False, \
        denoise_classification_map = True, morph_open = 100, morph_close = 20, \
        dilation_size = 151, \
        z_score = True, \
        limit_outliers = False, \
        extents = [dataset.bounds.left, dataset.bounds.right, 
                   dataset.bounds.bottom, dataset.bounds.top], \
        export_histogram = False, \
        export_heatmap = False, \
        half_normal_dist = True, \
        imagery_id = filename, \
        plotting = True, \
        verbose = True)

In [993]:
file_pattern = directory + 'tiles/*.tif'
for filename in glob.glob(file_pattern):
    print(os.path.basename(filename))
    
    if 'CLASSIFICATION_MAP' in filename:
        continue
    
    try:
        dataset = rasterio.open(filename)
        red,green,blue = dataset.read(1), dataset.read(2), dataset.read(3)
    except:
        continue
        
    img = np.zeros((blue.shape[0], blue.shape[1], 3), dtype = np.uint8)
    img[:,:,0] = blue
    img[:,:,1] = green
    img[:,:,2] = red

    density, classification_image, classification_confidence, spray_region = \
        imagery_statistical_binary_classification(\
            img, \
            [weed_1, weed_2, weed_3, weed_4, weed_5, weed_6, weed_7, \
            weed_8, weed_9, weed_10, weed_11, weed_12], \
            [crop_1, crop_2, crop_3, crop_4, crop_5, crop_6, crop_7, crop_8, \
            crop_9, crop_10], \
            image_resize_ratio = 5, \
            image_blur_kernel_size = (17,17), \
            template_blur_kernel_size = (3,3), \
            template_resize_size = 64, \
            duplicate_tolerance = 5, \
            common_tolerance = 5, \
            n_sigma_thresholds = [3.], \
            equalise = False, \
            denoise_classification_map = True, morph_open = 100, morph_close = 20, \
            dilation_size = 151, \
            z_score = True, \
            limit_outliers = False, \
            export_histogram = False, \
            export_heatmap = False, \
            half_normal_dist = True, \
            imagery_id = filename, \
            plotting = False, \
            verbose = True)

    # Export spray map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_SPRAY_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(spray_region == 0, 0, 255))

    # Export weed map to GeoTIFF.
    output_filename = filename.replace('.tif',f'_WEED_MAP.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, np.where(classification_confidence == 0, 0, 255))

    # Export classification CI to GeoTIFF.
    output_filename = filename.replace('.tif',f'_CLASSIFICATION_CONFIDENCE.tif')

    kwargs = dataset.meta
    kwargs.update(
        dtype=rasterio.uint8,
        count=1,
        compress='lzw')

    with rasterio.open(output_filename, 'w', **kwargs) as dst:
        dst.write_band(1, classification_confidence)

260324_Whealby_Wst_75pc_tile_99_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.8150
  362156 [px] in 3.00 CI, 0.4470 [%] of all pixels. 0.4470 [%] cumulative total.
Sample not detected in 99.55 [%] of all pixels.
Spray region is 3.61 [%] of total region.
260324_Whealby_Wst_75pc_tile_103_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186


/usr/local/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


mean = 0.0000, std = nan
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
260324_Whealby_Wst_75pc_tile_24_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 0.9532
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
260324_Whealby_Wst_75pc_tile_114_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.2241
  634560 [px] in 3.00 CI, 0.7832 [%] of all pixels. 0.7832 [%] cumulative total.
Sample not detected in 99.22 [%] of all pixels.
Spray region is 6.32 [%] of total region.
260324_Whealby_Wst_75pc_tile_33_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sa

Spray region is 1.57 [%] of total region.
260324_Whealby_Wst_75pc_tile_163_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = nan
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
260324_Whealby_Wst_75pc_tile_53_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 0.7507
   81758 [px] in 3.00 CI, 0.1009 [%] of all pixels. 0.1009 [%] cumulative total.
Sample not detected in 99.90 [%] of all pixels.
Spray region is 0.76 [%] of total region.
260324_Whealby_Wst_75pc_tile_139_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.0935
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] 

number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.2024
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
260324_Whealby_Wst_75pc_tile_60_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.7462
  541506 [px] in 3.00 CI, 0.6684 [%] of all pixels. 0.6684 [%] cumulative total.
Sample not detected in 99.33 [%] of all pixels.
Spray region is 2.53 [%] of total region.
260324_Whealby_Wst_75pc_tile_5_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.2516
  420849 [px] in 3.00 CI, 0.5194 [%] of all pixels. 0.5194 [%] cumulative total.
Sample not detected in 99.48 [%] of all pixels.
Spray region is 4.26 [%] of total region.
260324_Whealby_Wst_75pc_tile_150

mean = 0.0000, std = 1.7763
  329593 [px] in 3.00 CI, 0.4068 [%] of all pixels. 0.4068 [%] cumulative total.
Sample not detected in 99.59 [%] of all pixels.
Spray region is 3.47 [%] of total region.
260324_Whealby_Wst_75pc_tile_123_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.7597
  363209 [px] in 3.00 CI, 0.4483 [%] of all pixels. 0.4483 [%] cumulative total.
Sample not detected in 99.55 [%] of all pixels.
Spray region is 3.83 [%] of total region.
260324_Whealby_Wst_75pc_tile_97_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.7910
  216169 [px] in 3.00 CI, 0.2668 [%] of all pixels. 0.2668 [%] cumulative total.
Sample not detected in 99.73 [%] of all pixels.
Spray region is 1.65 [%] of total region.
260324_Whealby_Wst_75pc_tile_80_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in s

Spray region is 0.00 [%] of total region.
260324_Whealby_Wst_75pc_tile_85_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.7426
  274991 [px] in 3.00 CI, 0.3394 [%] of all pixels. 0.3394 [%] cumulative total.
Sample not detected in 99.66 [%] of all pixels.
Spray region is 2.78 [%] of total region.
260324_Whealby_Wst_75pc_tile_16_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.9226
  284570 [px] in 3.00 CI, 0.3512 [%] of all pixels. 0.3512 [%] cumulative total.
Sample not detected in 99.65 [%] of all pixels.
Spray region is 1.73 [%] of total region.
260324_Whealby_Wst_75pc_tile_131_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.0800
  222623 [px] in 3.00 CI, 0.2748 [%] of all pixels. 0.2748 [%] cumulative total.
Sample not detected in 99.73 [%] 

number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.8482
  212690 [px] in 3.00 CI, 0.2625 [%] of all pixels. 0.2625 [%] cumulative total.
Sample not detected in 99.74 [%] of all pixels.
Spray region is 1.89 [%] of total region.
260324_Whealby_Wst_75pc_tile_168_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = nan
       0 [px] in 3.00 CI, 0.0000 [%] of all pixels. 0.0000 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
260324_Whealby_Wst_75pc_tile_58_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.8739
  143527 [px] in 3.00 CI, 0.1771 [%] of all pixels. 0.1771 [%] cumulative total.
Sample not detected in 99.82 [%] of all pixels.
Spray region is 1.09 [%] of total region.
260324_Whealby_Wst_75pc_tile_151_

mean = 0.0000, std = 1.9409
  247766 [px] in 3.00 CI, 0.3058 [%] of all pixels. 0.3058 [%] cumulative total.
Sample not detected in 99.69 [%] of all pixels.
Spray region is 1.88 [%] of total region.
260324_Whealby_Wst_75pc_tile_138_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.4368
      89 [px] in 3.00 CI, 0.0001 [%] of all pixels. 0.0001 [%] cumulative total.
Sample not detected in 100.00 [%] of all pixels.
Spray region is 0.00 [%] of total region.
260324_Whealby_Wst_75pc_tile_106_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in sample B is 186
mean = 0.0000, std = 1.8741
  160871 [px] in 3.00 CI, 0.1986 [%] of all pixels. 0.1986 [%] cumulative total.
Sample not detected in 99.80 [%] of all pixels.
Spray region is 0.54 [%] of total region.
260324_Whealby_Wst_75pc_tile_21_EQUALISED.tif
number of distinct spectra in sample A is 2811
number of distinct spectra in